# bmondata Usage Examples

## Installation

The package can be installed through use of pip:

    pip install --upgrade bmondata
    
The `--upgrade` flag ensures that the newest version will replace a bmondata version
that you may already have.

## Use of the Server Class

The Server class is used to initiate requests to one BMON Server.  The base URL of the
BMON server is the one required parameter when instatiating the object.  If you want
to use the Server object to *store* sensor readings on the server, you also need
to provide the 'store_key' parameter.  The 'store_key' is the secret key found in the 
settings.py file on the BMON server.

In [2]:
import bmondata

# Make a Server object for retrieving data only
server = bmondata.Server('https://bmon.analysisnorth.com')

# The Server object below can also be used to store sensor readings 
# on the BMON server.
server = bmondata.Server('https://bmon.analysisnorth.com', store_key='temporary-key')

## Retrieve Sensor Readings

The `sensor_readings()` method is used to retrieve sensor readings from one or
more sensors.  The readings are returned in a Pandas DataFrame.  To retrieve
readings from one sensor, specify the Sensor ID as the first parameter:

In [3]:
df = server.sensor_readings('phil_hp_pwr_10187_temp')
df.head()

,phil_hp_pwr_10187_temp
2018-01-20 14:53:38,26.600
2018-01-20 14:56:50,26.600
2018-01-20 14:59:42,26.582
2018-01-20 16:28:59,25.777
2018-01-20 16:32:11,25.880


To retrieve readings from multiple sensors, provide a list of Sensor IDs:

In [4]:
df = df = server.sensor_readings(['phil_hp_pwr_10187_temp', 'phil_hp_pwr_7470_temp'])
df.head()

,phil_hp_pwr_10187_temp,phil_hp_pwr_7470_temp
2018-01-20 14:53:38,26.600,64.940
2018-01-20 14:56:32,NaN,64.958
2018-01-20 14:56:50,26.600,NaN
2018-01-20 14:59:42,26.582,65.120
2018-01-20 16:28:59,25.777,NaN


Readings from multiple sensors often are not synchronized in time, thus the DataFrame
will include many NaN values.  Time-averaging of readings is discussed later and
can eliminate most of the NaN values.

You can have the DataFrame use more meaningful column names by providing a column
label for one or more of the sensors:

In [5]:
sensors = [
    ('phil_hp_pwr_10187_temp', 'outdoor_temp'), 
    'phil_hp_pwr_7470_temp'
]

df = server.sensor_readings(sensors)
df.head()

,outdoor_temp,phil_hp_pwr_7470_temp
2018-01-20 14:53:38,26.600,64.940
2018-01-20 14:56:32,NaN,64.958
2018-01-20 14:56:50,26.600,NaN
2018-01-20 14:59:42,26.582,65.120
2018-01-20 16:28:59,25.777,NaN


For each Sensor that you wish to label, use a two-tuple containing the Sensor ID
and the Sensor Label instead of just supplying the Sensor ID.

To filter the readings based on date/time, use the `start_ts` and `end_ts` parameters:

In [6]:
df = server.sensor_readings(
    sensors,
    start_ts = '2019-01-15 3:00 pm',
    end_ts = '2019-01-17 10:30 am'
)
df.head()

,outdoor_temp,phil_hp_pwr_7470_temp
2019-01-15 15:00:20,NaN,70.58
2019-01-15 15:00:21,33.04,NaN
2019-01-15 15:02:23,33.04,NaN
2019-01-15 15:02:31,NaN,70.58
2019-01-15 15:04:01,NaN,70.58


The format of `start_ts` and `end_ts` is very flexible.  Any date/time that can be
parsed by dateutil.parser.parse() will work.  If `start_ts` is not provided, readings
start at the earliest available; if `end_ts` is not provided, readings continue through
the latest available.

You can request that the sensor readings be averaged into time periods such as 1 hour or
1 day.  For a full list of the possible time period codes, see 
[DateOffset Objects](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).
Here is an example for 1 hour averaging:

In [7]:
df = server.sensor_readings(
    sensors,
    start_ts = '2019-01-15 3:00 pm',
    end_ts = '2019-01-17 10:30 am',
    averaging = '1H'
)
df.head()

,outdoor_temp,phil_hp_pwr_7470_temp
2019-01-15 15:00:00,33.006233,70.139767
2019-01-15 16:00:00,32.436133,70.168467
2019-01-15 17:00:00,31.556200,70.931667
2019-01-15 18:00:00,31.355200,70.568900
2019-01-15 19:00:00,31.684400,70.030333


The default is to label the time period with the time at left (beginning) edge of the
interval.  If instead you want the timestamp to fall at a different point in the
interval, you can use the `label_offset` parameter to shift it.  Here we mark the
middle of the interval by using an offset of 30 minutes:

In [8]:
df = server.sensor_readings(
    sensors,
    start_ts = '2019-01-15 3:00 pm',
    end_ts = '2019-01-17 10:30 am',
    averaging = '1H',
    label_offset = '30min'
)
df.head()

,outdoor_temp,phil_hp_pwr_7470_temp
2019-01-15 15:30:00,33.006233,70.139767
2019-01-15 16:30:00,32.436133,70.168467
2019-01-15 17:30:00,31.556200,70.931667
2019-01-15 18:30:00,31.355200,70.568900
2019-01-15 19:30:00,31.684400,70.030333


## Retrieve Sensor Metadata

Sensor titles, units and other information can be retrieved for one or more
sensors by using the `sensors()` method.  Pass a Sensor ID or a list of Sensor
IDs to the method:

In [9]:
server.sensors(['phil_hp_pwr_10187_temp', 'phil_hp_pwr_7470_temp'])

[{'id': 234,
  'sensor_id': 'phil_hp_pwr_10187_temp',
  'title': 'New Outdoor Wireless Temp',
  'notes': 'No sensor notes available.',
  'is_calculated': False,
  'tran_calc_function': 'val - 0.76',
  'function_parameters': '',
  'calculation_order': 0,
  'formatting_function': '',
  'other_properties': '',
  'unit': 'deg F',
  'buildings': [{'bldg_id': 5, 'sensor_group': 'Weather', 'sort_order': 40}]},
 {'id': 243,
  'sensor_id': 'phil_hp_pwr_7470_temp',
  'title': 'House Temp',
  'notes': 'No sensor notes available.',
  'is_calculated': False,
  'tran_calc_function': 'val + 0.24',
  'function_parameters': '',
  'calculation_order': 0,
  'formatting_function': '',
  'other_properties': '',
  'unit': 'deg F',
  'buildings': [{'bldg_id': 5,
    'sensor_group': 'Space Conditions, Temperature',
    'sort_order': 10}]}]

The return value is a list of dictionaries, each dictionary describing a Sensor.
The keys in the dictionary are the fields associated with the Sensor model in the
BMON Django application. The `buildings` key in the dictionary gives a list of
buildings that the Sensor is associated with.
Further documentation of the fields is available
[Here](https://github.com/alanmitchell/bmon/blob/master/bmsapp/models.py); search
for the `class Sensor` section of the code.

If you do not provide any IDs (either no parameters, or an empty list), information
for *all* sensors will be returned.  For example, `server.sensors()` will return
a list of all sensors.

## Building and Organization Information

Methods are available to return information about Buildings and Organizations in the
BMON system.  Pass one or a list of Building IDs to get Building information:

In [17]:
server.buildings([6, 13])

[{'id': 6,
  'title': 'Rivendell Hall',
  'report_footer': '',
  'latitude': 64.864072,
  'longitude': -147.775014,
  'timezone': 'US/Alaska',
  'schedule': 'M-F: 8a-5p',
  'timeline_annotations': '',
  'other_properties': '',
  'current_mode': '',
  'sensors': [{'sensor_id': 'pafa_temp',
    'sensor_group': 'Weather',
    'sort_order': 10},
   {'sensor_id': 'pafa_wind', 'sensor_group': 'Weather', 'sort_order': 20},
   {'sensor_id': 'pioneer_temp', 'sensor_group': 'Weather', 'sort_order': 30},
   {'sensor_id': 'pioneer_wind', 'sensor_group': 'Weather', 'sort_order': 40},
   {'sensor_id': 'nortech_PV_power',
    'sensor_group': 'Solar PV',
    'sort_order': 10},
   {'sensor_id': '3c0034000447343233323032_temp',
    'sensor_group': 'IAQ Monitor #1',
    'sort_order': 10},
   {'sensor_id': '3c0034000447343233323032_rh',
    'sensor_group': 'IAQ Monitor #1',
    'sort_order': 20},
   {'sensor_id': '3c0034000447343233323032_co2',
    'sensor_group': 'IAQ Monitor #1',
    'sort_order': 30},


The `sensors` item gives a list of Sensors associated with the Building.
Further documentation of the fields is available
[Here](https://github.com/alanmitchell/bmon/blob/master/bmsapp/models.py); search
for the `class Building` section of the code.
If you do not provide any IDs (either no parameters, or an empty list), information
for *all* buildings will be returned.

Here is the method for retrieving information about Organizations:

In [11]:
server.organizations([1, 2])

[{'id': 1,
  'title': 'Homes',
  'sort_order': 10,
  'buildings': [[16, "Chris's House"],
   [8, "Dustin's Neighborhood"],
   [4, 'Homer Strawbale'],
   [14, "Ian's House"],
   [5, 'Kaluza House'],
   [3, 'Mitchell House'],
   [7, 'Rehfeldt Home'],
   [19, "Tyler's House"]]},
 {'id': 2,
  'title': 'THRHA',
  'sort_order': 20,
  'buildings': [[18, 'THRHA Angoon Housing'],
   [17, 'THRHA Juneau Warehouse'],
   [2, 'THRHA Kake Senior Center']]}]

The `buildings` key gives the list of buildings associated with the organization.
Again, further documentation of the fields is available
[Here](https://github.com/alanmitchell/bmon/blob/master/bmsapp/models.py); search
for the `class Organization` section of the code. `server.organizations()` will return
information on all Organizations.

## Storing New Sensor Readings

The `bmondata` package can be used to store new sensor readings into
the BMON server's sensor reading database.  Readings are stored using the Server
object, and a list of new readings are provided.  Here is an example:

In [15]:
import time
server.store_sensor_readings([
    (time.time(), '_testing', 18.8),
    (time.time(), '_hello', 24.3),
])

2

### Used to Save this Notebook as an HTML File

In [18]:
!jupyter nbconvert usage_examples.ipynb --to html

[NbConvertApp] Converting notebook usage_examples.ipynb to html
[NbConvertApp] Writing 304859 bytes to usage_examples.html
